# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = r"C:\Users\skm72\Desktop\git\Homework\python-api-challenge\python-api-challenge\WeatherPy\output_data\cities.csv"
weather_df=pd.read_csv(csv_file)
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bauchi,10.50,10.00,72.97,88,96,3.18,NG,1597791092
1,1,Mahébourg,-20.41,57.70,71.60,73,40,14.99,MU,1597791093
2,2,Elmira,42.09,-76.81,75.20,38,40,4.70,US,1597791094
3,3,Hobart,-42.88,147.33,44.01,87,75,9.17,AU,1597790825
4,4,Sobolevo,54.43,31.90,59.32,82,99,8.39,RU,1597791097


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"]

In [4]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
clean_weather_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

clean_weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
120,120,Zaozyornoye,45.16,33.28,71.10,64,0,2.89,UA,1597791230
158,158,Sinnamary,5.38,-52.95,79.11,83,0,8.55,GF,1597791274
225,225,Ol'ga,43.74,135.29,79.21,78,0,3.49,RU,1597791351
249,249,Kiryat Gat,31.61,34.76,77.00,77,0,2.68,IL,1597791380
282,282,Tūkrah,32.53,20.58,75.40,86,0,1.92,LY,1597791417


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = clean_weather_df.loc[:,["City","Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
120,Zaozyornoye,UA,45.16,33.28,
158,Sinnamary,GF,5.38,-52.95,
225,Ol'ga,RU,43.74,135.29,
249,Kiryat Gat,IL,31.61,34.76,
282,Tūkrah,LY,32.53,20.58,


In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
print("-------End of Search-------")

Retrieving Results for Index 120: Zaozyornoye.
Closest hotel in Zaozyornoye is Gostinyy Dvor "Voyazh".
------------
Retrieving Results for Index 158: Sinnamary.
Closest hotel in Sinnamary is L'oiseau de Paradis.
------------
Retrieving Results for Index 225: Ol'ga.
Closest hotel in Ol'ga is Gostinitsa Rosmorport..
------------
Retrieving Results for Index 249: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
------------
Retrieving Results for Index 282: Tūkrah.
Closest hotel in Tūkrah is مطعم الحيطة.
------------
Retrieving Results for Index 311: Fort-Shevchenko.
Closest hotel in Fort-Shevchenko is Oasis hotel Bautino.
------------
Retrieving Results for Index 372: Murgab.
Missing field/result... skipping.
------------
Retrieving Results for Index 429: Caravelas.
Closest hotel in Caravelas is Mar Doce de Abrolhos.
------------
Retrieving Results for Index 468: Methóni.
Closest hotel in Methóni is Methoni Beach Hotel.
------------
Retrieving Results for Index 485: Dāmghān.
Closes

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))